# Building a bigram and using 100YearOfSolitute as dataset 

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


/Users/ashutosh/Codes/Projects/LLM-From-Scratch/.envllm/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [8]:
with open("100YearOfSolitude.txt", 'r', encoding='utf-8') as f:
    text = f.read()

char = sorted(set(text))
vocab_size = len(chars)

# print(char)
# print(vocab_size)

In [11]:
string_to_int = {ch:i for i, ch in enumerate(char)}
int_to_string = {i:ch for i, ch in enumerate(char)}

encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l:[int_to_string[i] for i in l]

data = torch.tensor(encode(text), dtype=torch.long)

# print(data[:50])

In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 42, 52, 65, 71, 52,  1, 42],
        [70,  1, 63, 52, 71, 56, 69,  7],
        [59, 60, 65, 62, 60, 65, 58,  1],
        [57, 66, 69,  1, 59, 56, 69,  1]])
targets:
tensor([[42, 52, 65, 71, 52,  1, 42, 66],
        [ 1, 63, 52, 71, 56, 69,  7,  1],
        [60, 65, 62, 60, 65, 58,  1, 52],
        [66, 69,  1, 59, 56, 69,  1, 60]])


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

['\n', 'd', '6', 'N', ':', 'O', '!', ':', 'I', '&', 'A', '5', '_', ';', 'a', 'R', 'O', 'Q', 'L', '_', 'D', '&', 'f', 'v', '1', '-', 'A', 'H', 'c', 'f', 'A', 'M', '6', '•', 'N', 'u', 'h', 'A', '©', '—', '.', 'C', ':', 'i', '4', 'J', 'y', 'd', 'b', '9', '“', '“', 'j', '4', 'B', '9', 'z', 'A', '¬', 'g', 's', '/', '3', '„', '6', '8', 'A', '“', 'I', 'W', 'o', '’', '-', '9', 'V', 't', 'T', '©', 'P', '5', 'h', '„', '(', '1', '\n', 'F', '„', '/', 'E', 'x', 'h', ',', 'T', 'k', 'a', 't', 'w', ';', '!', 'u', '8', 'w', 'p', '0', '9', 'W', 'L', 'g', 'v', 'A', '”', 'E', 'l', 'Z', 'l', '^', 'v', 'f', 'D', 'R', '!', '8', '!', 'p', '7', 'n', 'C', 'F', '3', '¬', '“', 'w', '_', 'D', 'Z', '/', ',', 'P', 'I', 't', 'G', 'S', 'a', 'O', '(', 'p', '“', 'e', 'Q', '•', 'H', 'E', '-', 'a', 'k', "'", 'T', 'c', 'b', 'i', 'K', '-', 'G', '^', 'n', '2', 'b', 'l', "'", '/', 'j', '/', '\n', ' ', 'M', 'w', 'M', '•', 'f', '/', '¬', 'W', '?', 'W', '3', 'M', 'O', 'U', 'c', ';', '5', 'x', 'c', 'f', '“', '0', 'v', 'S', '7', '

In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.939, val loss: 4.932
step: 250, train loss: 4.844, val loss: 4.878
step: 500, train loss: 4.792, val loss: 4.795
step: 750, train loss: 4.718, val loss: 4.722
5.008190631866455


In [16]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

['\n', 'i', '?', '’', 'c', 'o', 'B', '—', 'H', 'e', 'a', '”', 'N', 'o', '“', ' ', 'W', 'x', '’', 'o', '1', '6', 'U', '3', '„', '“', 'V', 'h', 'p', 'H', 'A', '0', '—', 'A', '“', 'v', 'S', '5', 'Z', 'W', '7', '”', 'w', '_', ')', 'U', 's', ',', '¬', '-', '1', '/', 'G', 'l', 't', '•', 'B', 'M', '0', 'D', 'F', 'V', 'f', 'H', 'P', '!', 'm', '1', '\n', 'g', '8', 'q', 'i', 'n', 'q', 'i', '}', '7', 'y', 't', '3', 'K', 'J', '•', 'e', 'Y', 'j', 'q', 'y', 'p', 'c', '2', 'e', 'y', 'W', 'I', 'P', '6', 'i', '©', 'z', 'F', 'v', 'g', 'e', '!', 'c', 'f', 'F', 'N', '’', '•', 'p', '“', 'H', '^', 'h', 'G', 'p', '“', 'm', '\n', '“', 'e', '\n', 'S', '5', 'h', '„', 'B', 'V', 'X', '4', 'J', 'M', 'g', 'A', 'd', '}', 'a', 'b', 'l', '¬', 'c', '¬', 't', ')', 'G', 'I', 't', ';', '•', 'e', 't', 'e', '9', 'i', '—', '&', ';', 'g', 'N', '.', '9', '¬', 'k', 'l', 'G', 'c', 't', '•', '&', 'y', '7', 'J', 'S', 'M', '8', 'N', 'h', '^', 'C', '/', '(', 'v', '!', '3', 'g', 'y', 'P', '\n', '¬', 'p', 'l', '’', 'P', 'z', 'c', '3',